# Домашня робота

## 1. Робота з не проектинвними деревами

https://www.aclweb.org/anthology/P09-1040.pdf

Swap(σ|wi|wj, β, Ac) = (σ|wj, wi|β, Ac)

- Projectivity is not a property of a dependency tree in itself, but of the tree in combination with a word order, and that a tree can always be made projective byre ordering the nodes
- projective order is defined by an inorder traversal of set of nodes
- the oracle predicts LEFT or RIGHT if the two top nodes on the stack should be connected by an arc and if the dependent node of this arc is already connected to all its dependents
- it predicts SWAP if the two top nodes are not in projective order
- it is important to note that SWAP is only permissible when the two nodes on top of the stack are in the original word order, which prevents the same two nodes from beings wapped more than once, and when the left most node is distinct from the root node 0

In [1]:
from utils import load_treebank, render_tree, golden_rels, ROOT, uas_report, intersect

def clean_trees(trees):
    return [[t for t in tree if type(t['id'])==int] for tree in trees]

train_trees = clean_trees(load_treebank('uk_iu-ud-train.conllu'))
test_trees = clean_trees(load_treebank('uk_iu-ud-dev.conllu'))

In [4]:
from collections import defaultdict

SHIFT = 'shift'
SWAP = 'swap'
RIGHT = 'right'
LEFT = 'left'

def no_children(node, stack, queue):
    for n in stack:
        if n['head'] == node['id']:
            return False
        
    for n in queue:
        if n['head'] == node['id']:
            return False    
        
    return True

def is_left(s, q):
    return s[-2]['id'] > 0 and \
            s[-2]['head'] == s[-1]['id'] and \
            no_children(s[-2], s, q)

def is_right(s, q):
    return s[-1]['head'] == s[-2]['id'] and \
            no_children(s[-1], s, q)

def is_swap(s, porder):
    return s[-2]['id'] > 0 and porder.index(s[-1]['id']) < porder.index(s[-2]['id'])
    
    
def oracle(stack, queue, relations, porder):
    if len(stack) > 1:                        
        if is_left(stack, queue):
            return LEFT
        elif is_right(stack, queue):
            return RIGHT
        elif is_swap(stack, porder):
            return SWAP
        else:
            return SHIFT
    else:
        return SHIFT

    
def apply(action, stack, queue, relations):
    if action == SHIFT:
        stack.append(queue.pop(0))
    elif action == RIGHT:
        relations.append((stack[-1]['id'], stack[-2]['id']))
        del stack[-1]
    elif action == LEFT:
        relations.append((stack[-2]['id'], stack[-1]['id']))
        del stack[-2]
    elif action == SWAP:
        queue.insert(0, stack[-2])
        del stack[-2]
    else:
        raise Exception('Bad action')

def in_order_traverse(tokens, start=0):
    for x in tokens[start]: 
        if x < start: 
            yield from in_order_traverse(tokens, x) 
    yield start 
    for x in tokens[start]: 
        if x > start: 
            yield from in_order_traverse(tokens, x)

def proj_order(queue):
    deps = defaultdict(list)
    for node in queue:
        deps[node['head']].append(node['id'])
    return list(in_order_traverse(deps))


def traverse(tree, log=False):
    stack, queue, relations = [ROOT], tree[:], []
    
    porder = proj_order(queue)

    while len(stack) > 1 or queue: 
        action = oracle(stack, queue, relations, porder)
        
        if log:
            print('Action:', action.upper())
            print('Stack:', [i['form'] + '-' + str(i['id']) for i in stack])
            print('Queue:', [i['form'] + '-' + str(i['id']) for i in queue])
            print('Relations:', relations)
            print()
        
        yield(stack, queue, relations, action)

        apply(action, stack, queue, relations)

        
def relations(tree):    
    for _, _, relations, _ in traverse(tree, log=False): pass
    return relations

In [5]:
def is_projective(tree):
    return proj_order(tree)[1:] == [node['id'] for node in tree]

non_proj_trees = list(filter(lambda t: not is_projective(t), train_trees))

print('Total:', len(train_trees))
print('Projective:', len(train_trees) - len(non_proj_trees))
print('Non-projective:', len(non_proj_trees))

Total: 5496
Projective: 5057
Non-projective: 439


In [6]:
# tree = non_proj_trees[70]
tree = non_proj_trees[1]

In [7]:
render_tree(tree)

In [8]:
golden = sorted([(node['id'], node['head']) for node in tree])
parsed = sorted(relations(tree))
match = intersect(golden, parsed)

print('Golden:', golden)
print('Parsed:', parsed)
print('Correct: {}/{}'.format(match, len(golden)))

Golden: [(1, 3), (2, 1), (3, 0), (4, 5), (5, 3), (6, 12), (7, 12), (8, 10), (9, 10), (10, 13), (11, 12), (12, 3), (13, 12), (14, 15), (15, 13), (16, 19), (17, 19), (18, 19), (19, 15), (20, 19), (21, 23), (22, 23), (23, 15), (24, 23), (25, 23), (26, 27), (27, 15), (28, 29), (29, 27), (30, 15), (31, 30), (32, 3)]
Parsed: [(1, 3), (2, 1), (3, 0), (4, 5), (5, 3), (6, 12), (7, 12), (8, 10), (9, 10), (10, 13), (11, 12), (12, 3), (13, 12), (14, 15), (15, 13), (16, 19), (17, 19), (18, 19), (19, 15), (20, 19), (21, 23), (22, 23), (23, 15), (24, 23), (25, 23), (26, 27), (27, 15), (28, 29), (29, 27), (30, 15), (31, 30), (32, 3)]
Correct: 32/32


In [9]:
[(node['id'], node['head']) for node in tree]

[(1, 3),
 (2, 1),
 (3, 0),
 (4, 5),
 (5, 3),
 (6, 12),
 (7, 12),
 (8, 10),
 (9, 10),
 (10, 13),
 (11, 12),
 (12, 3),
 (13, 12),
 (14, 15),
 (15, 13),
 (16, 19),
 (17, 19),
 (18, 19),
 (19, 15),
 (20, 19),
 (21, 23),
 (22, 23),
 (23, 15),
 (24, 23),
 (25, 23),
 (26, 27),
 (27, 15),
 (28, 29),
 (29, 27),
 (30, 15),
 (31, 30),
 (32, 3)]

In [10]:
def extract_features(stack, queue, relations, *args):
    features = {}
    
    if stack:
        features['s0-form'] = stack[-1]['form']
        features['s0-lemma'] = stack[-1]['lemma']
        features['s0-tag'] = stack[-1]['upostag'] 
    if len(stack) > 1:
        features['s1-tag'] = stack[-2]['upostag']
        
    if queue:
        features['q0-form'] = queue[0]['form']
        features['q0-lemma']= queue[0]['lemma']
        features['q0-tag']= queue[0]['upostag']
    if len(queue) > 1:
        features['q1-form'] = queue[1]['form']
        features['q1-tag'] = queue[1]['upostag']
    if len(queue) > 2:
        features['q2-tag'] = queue[2]['upostag']
    if len(queue) > 3:
        features['q3-tag'] = queue[3]['upostag']
    
    return features

def create_get_data(extractor):
    def get_data(tree):
        features, labels = [], []

        for stack, queue, relations, action in traverse(tree):
            features.append(extractor(stack, queue, relations, action))
            labels.append(action)

        return features, labels
    return get_data

def prepare_data(trees, extractor):
    error_ids = []
    features, labels = [], []
    
    for id, tree in enumerate(trees):
        try:
            f, l = extractor([t for t in tree if type(t['id'])==int])
            features += f
            labels += l  
        except Exception as e:
            error_ids.append(id)
            raise(e)
            pass
    
    if error_ids:
        raise Exception('Error ids: ' + str(error_ids))

    return features, labels

extractor = create_get_data(extract_features)

train_features, train_labels = prepare_data(train_trees, extractor)
print(len(train_features), len(train_labels))

test_features, test_labels = prepare_data(test_trees, extractor)  
print(len(test_features), len(test_labels))

189432 189432
25628 25628


In [11]:
from collections import Counter
print(Counter(train_labels))
print(Counter(test_labels))

Counter({'shift': 94716, 'left': 48875, 'right': 43526, 'swap': 2315})
Counter({'shift': 12814, 'left': 6463, 'right': 6111, 'swap': 240})


❗ Прикладів `swap` зовсім мало. Схоже, будуть проблеми з моделлю.

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import confusion_matrix, classification_report

def report(test, pred):
    print(confusion_matrix(test, pred))
    print()
    print(classification_report(test, pred))
    
def run_model(X_train, y_train, X_test, y_test):
    vectorizer = DictVectorizer()
    vec = vectorizer.fit(X_train)

    print('Total number of features:', len(vec.get_feature_names()))

    X_train = vec.transform(X_train)
    X_test = vec.transform(X_test)

    lrc = LogisticRegression(multi_class="multinomial", verbose=1)
    lrc.fit(X_train, y_train)

    y_pred = lrc.predict(X_test)    

    report(y_test, y_pred)
    
    return lrc, vec

In [13]:
run_model(train_features, train_labels, test_features, test_labels)

Total number of features: 111126


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/h8si2w8/.pyenv/versions/3.8.2/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   17.8s finished


[[ 5320   190   950     3]
 [  320  4526  1263     2]
 [ 1005   723 11080     6]
 [   38    42   153     7]]

              precision    recall  f1-score   support

        left       0.80      0.82      0.81      6463
       right       0.83      0.74      0.78      6111
       shift       0.82      0.86      0.84     12814
        swap       0.39      0.03      0.05       240

    accuracy                           0.82     25628
   macro avg       0.71      0.61      0.62     25628
weighted avg       0.81      0.82      0.81     25628



(LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='multinomial', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=1,
                    warm_start=False),
 DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
                sparse=True))

## 2. Підбір ознак

😕 `swap` передбачається ще гірше ніж я очікував. очевидно, що замало прикладів. також, думаю, ознаки з практичної роботи не репрезентативні для нового парсера. можливо щось вдасться покращити!

In [14]:
def extract_features_v2(stack, queue, relations, *args):
    features = dict()
    
    if len(stack) > 1:        
        features['s0-form'] = stack[-1]['form']
        features['s0-lemma'] = stack[-1]['lemma']
        features['s0-tag'] = stack[-1]['upostag']
        if 'feats' in stack[-1] and stack[-1]['feats']:
            for k, v in stack[-1]['feats'].items():
                features['s0-' + k] = v
           
        features['s1-form'] = stack[-2]['form']
        features['s1-lemma'] = stack[-2]['lemma']
        features['s1-tag'] = stack[-2]['upostag']
        if 'feats' in stack[-2] and stack[-2]['feats']:
            for k, v in stack[-2]['feats'].items():
                features['s1-' + k] = v
    
    if len(stack) > 2:
        features['s2-form'] = stack[-3]['form']
        features['s2-tag'] = stack[-3]['upostag']
    
    if len(stack) > 3:
        features['s3-tag'] = stack[-4]['upostag']
        
    if len(stack) > 4:
        features['s4-tag'] = stack[-5]["upostag"]
    
    if len(queue) > 0:        
        features['q0-form'] = queue[0]['form']
        features['q0-tag'] = queue[0]['upostag']
    
    if len(queue) > 1:
        features['q1-tag'] = queue[1]['upostag']
    
    if len(queue) > 2:
        features['q2-tag'] = queue[2]['upostag']
    
    return features

extractor = create_get_data(extract_features_v2)

train_features, train_labels = prepare_data(train_trees, extractor)
test_features, test_labels = prepare_data(test_trees, extractor)  

run_model(train_features, train_labels, test_features, test_labels)

Total number of features: 117917


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/h8si2w8/.pyenv/versions/3.8.2/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.4s finished


[[ 5715   132   611     5]
 [  254  4726  1122     9]
 [  817   780 11194    23]
 [   37    32   131    40]]

              precision    recall  f1-score   support

        left       0.84      0.88      0.86      6463
       right       0.83      0.77      0.80      6111
       shift       0.86      0.87      0.87     12814
        swap       0.52      0.17      0.25       240

    accuracy                           0.85     25628
   macro avg       0.76      0.67      0.70     25628
weighted avg       0.84      0.85      0.84     25628



(LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='multinomial', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=1,
                    warm_start=False),
 DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
                sparse=True))

In [15]:
def extract_features_v3(stack, queue, relations, *args):
    features = extract_features_v2(stack, queue, relations)
    
    if stack and queue:
        s0 = stack[-1]
        q0 = queue[0]
        
        features['s0+q0-form'] = s0['form'] + q0['form']
        features['s0+q0-form'] = s0['form'] + q0['form']
        features['s0+q0-tag'] = s0['upostag'] + q0['upostag']
        features['s0+q0-form-tag'] = s0['form'] + s0['upostag'] + \
                                     q0['form'] + q0['upostag']

    if len(stack) > 2:
        s0 = stack[-1]
        s1 = stack[-2]
        
        # bi-grams
        features['s0+s1-form'] = s0['form'] + s1['form']
        features['s0+s1-form'] = s0['form'] + s1['form']
        features['s0+s1-tag'] = s0['upostag'] + s1['upostag']
        features['s0+s1-form+tag'] = s0['form'] + s0['upostag'] + \
                                     s1['form'] + s1['upostag']
        
    return features

extractor = create_get_data(extract_features_v3)

train_features, train_labels = prepare_data(train_trees, extractor)
test_features, test_labels = prepare_data(test_trees, extractor)  

run_model(train_features, train_labels, test_features, test_labels)

Total number of features: 542461


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/h8si2w8/.pyenv/versions/3.8.2/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   40.1s finished


[[ 5933   116   404    10]
 [  175  5034   895     7]
 [  727   828 11231    28]
 [   32    24   134    50]]

              precision    recall  f1-score   support

        left       0.86      0.92      0.89      6463
       right       0.84      0.82      0.83      6111
       shift       0.89      0.88      0.88     12814
        swap       0.53      0.21      0.30       240

    accuracy                           0.87     25628
   macro avg       0.78      0.71      0.73     25628
weighted avg       0.87      0.87      0.87     25628



(LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='multinomial', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=1,
                    warm_start=False),
 DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
                sparse=True))

In [16]:
def extract_features_v4(stack, queue, relations, *args):
    features = extract_features_v3(stack, queue, relations)
    
    features['q-empty'] = not bool(queue) 
    
    if len(stack) > 1:
        features['distance'] = stack[-1]['id'] - stack[-2]['id'] 
       
        features['s0-rchildren-count'] = len([r for r in relations if r[1] == stack[-1]['id']])
        features['s0-lchildren-count'] = len([r for r in relations if r[0] == stack[-1]['id']])
        features['s1-rchildren-count'] = len([r for r in relations if r[1] == stack[-2]['id']])
        features['s1-lchildren-count'] = len([r for r in relations if r[0] == stack[-2]['id']])
        
    return features


extractor = create_get_data(extract_features_v4)

train_features, train_labels = prepare_data(train_trees, extractor)
test_features, test_labels = prepare_data(test_trees, extractor)  

model, vec = run_model(train_features, train_labels, test_features, test_labels)

Total number of features: 542467


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/h8si2w8/.pyenv/versions/3.8.2/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   41.0s finished


[[ 5945    91   400    27]
 [  101  5034   963    13]
 [  531   924 11328    31]
 [   28    11    98   103]]

              precision    recall  f1-score   support

        left       0.90      0.92      0.91      6463
       right       0.83      0.82      0.83      6111
       shift       0.89      0.88      0.88     12814
        swap       0.59      0.43      0.50       240

    accuracy                           0.87     25628
   macro avg       0.80      0.76      0.78     25628
weighted avg       0.87      0.87      0.87     25628



Що ж, вдалося покращити результат для `swap` з `0.39, 0.03, 0.05` до `0.59, 0.43, 0.50`

In [17]:
def dep_parse(tree):
    stack, queue, relations = [ROOT], tree[:], []

    while len(stack) > 1 or queue:
        features = extract_features_v4(stack, queue, relations)
        vec_features = vec.transform([features])
        action = model.predict(vec_features)[0]
        
        apply(action, stack, queue, relations)

    return sorted(relations)

In [19]:
from tqdm import tqdm

# remove trees casuse endless loop :(
trees = test_trees[:541] + test_trees[542:]

golden, predicted = [], []
for tree in tqdm(trees):
    golden.append([(node['id'], node['head']) for node in tree])
    predicted.append(dep_parse(tree))

100%|██████████| 671/671 [03:44<00:00,  2.99it/s]


In [20]:
uas_report(golden, predicted)

Total: 12549
Correct: 9280
UAS: 0.74


💭 Щож, вдалося досягти якогось покращення. Але все ж `swap` перебачається 

## 3. Підбір класифікатора і оптимізація гіперпараметрів

In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, cross_val_predict, cross_val_score

vectorizer = DictVectorizer()
vec = vectorizer.fit(train_features)

X_train, X_test = vec.transform(train_features), vec.transform(test_features)
y_train, y_test = train_labels, test_labels

grid_params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': range(1,20,2)
}
dtc_model = GridSearchCV(DecisionTreeClassifier(), grid_params, cv=5, verbose=5, n_jobs=-1)
dtc_model.fit(X_train, y_train)

final_model = dtc_model.best_estimator_
y_pred = final_model.predict(X_test)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 36.5min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 60.0min finished


In [27]:
report(y_test, y_pred)

[[ 5964    67   405    27]
 [   91  5048   964     8]
 [  361   906 11507    40]
 [   23     8    96   113]]

              precision    recall  f1-score   support

        left       0.93      0.92      0.92      6463
       right       0.84      0.83      0.83      6111
       shift       0.89      0.90      0.89     12814
        swap       0.60      0.47      0.53       240

    accuracy                           0.88     25628
   macro avg       0.81      0.78      0.79     25628
weighted avg       0.88      0.88      0.88     25628



In [28]:
def dep_parse_v2(tree):
    stack, queue, relations = [ROOT], tree[:], []

    while len(stack) > 1 or queue:
        features = extract_features_v4(stack, queue, relations)
        vec_features = vec.transform([features])
        action = final_model.predict(vec_features)[0]
        
        apply(action, stack, queue, relations)

    return sorted(relations)

golden, predicted = [], []
for tree in tqdm(test_trees):
    golden.append([(node['id'], node['head']) for node in tree])
    predicted.append(dep_parse_v2(tree))

100%|██████████| 672/672 [00:56<00:00, 11.97it/s]


In [29]:
uas_report(golden, predicted)

Total: 12574
Correct: 9065
UAS: 0.72


## 4. Тестування на нових реченнях

In [30]:
import pymorphy2
import tokenize_uk

morph = pymorphy2.MorphAnalyzer(lang='uk')

DET = ['будь-який', 'ваш', 'ввесь', 'весь', 'все', 'всенький', 'всякий',
       'всілякий', 'деякий', 'другий', 'жадний', 'жодний', 'ин.', 'ін.',
       'інакший', 'інш.', 'інший', 'їх', 'їхній', 'її', 'його', 'кожний',
       'кожній', 'котрий', 'котрийсь', 'кілька', 'мій', 'наш', 'небагато',
       'ніякий', 'отакий', 'отой', 'оцей', 'сам', 'самий', 'свій', 'сей',
       'скільки', 'такий', 'тамтой', 'твій', 'те', 'той', 'увесь', 'усякий',
       'усілякий', 'це', 'цей', 'чий', 'чийсь', 'який', 'якийсь']

PREP = ["до", "на"]

mapping = {"ADJF": "ADJ", "ADJS": "ADJ", "COMP": "ADJ", "PRTF": "ADJ",
           "PRTS": "ADJ", "GRND": "VERB", "NUMR": "NUM", "ADVB": "ADV",
           "NPRO": "PRON", "PRED": "ADV", "PREP": "ADP", "PRCL": "PART"}

def normalize_pos(word):
    if word.tag.POS == "CONJ":
        if "coord" in word.tag:
            return "CCONJ"
        else:
            return "SCONJ"
    elif "PNCT" in word.tag:
        return "PUNCT"
    elif word.normal_form in PREP:
        return "PREP"
    elif word.normal_form in DET:
        return "DET"
    else:
        return mapping.get(word.tag.POS, word.tag.POS)
    
# TODO: animacy, gender, number, case, aspect, voice, person, tense
# TODO: check value correspondence 
def normalize_feats(w):
    return {}

def encode_word(w, id):
    m = morph.parse(w)[0]
    return {
        'id': id,
        'form': w,
        'lemma': m.normal_form,
        'upostag': normalize_pos(m)
    }

def prepare_sent(text):
    tokens = tokenize_uk.tokenize_words(text)
    return [encode_word(t, i) for i, t in enumerate(tokens, 1)]

def tree_from_text(text):
    tokens = prepare_sent(text)
    relations = dep_parse(tokens)

    for i, t in enumerate(tokens):
        t['head'] = relations[i][1]
        t['deprel'] = ''

    return tokens

In [31]:
render_tree(tree_from_text('Парубок стояв, як зачарований'))

In [32]:
render_tree(tree_from_text('За чисельністю населення Україна посідає восьме місце в Європі.'))

In [33]:
render_tree(tree_from_text("З неба, як розтоплене золото, ллється на землю блискучий світ сонця"))